# Gravity Model with PolityV Scores
---

### Basic Specification:
$$ y_{ijt} = \alpha + \beta_1 GDP_i + \beta_2 GDP_j + \beta_3 Dist_{ij} + \theta_t + \epsilon_{ijt} $$

### Modified Specification:
$$ y_{ijt} = \alpha + \beta_1 GDP_i + \beta_2 GDP_j + \beta_3 Dist_{ij} + \beta_4 PolityDist_{ijt} + \theta_t + \epsilon_{ijt} $$

In [1]:
import numpy as np
import math
import pandas as pd
from linearmodels import PanelOLS

## 1. Prepare data
---

In [2]:
data = pd.read_csv("../data/trade-polity.csv", index_col=0)
data

,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,gsp_rec,flow,validmirror,family,country_o,country_d,tradelink,polity_o,polity_d,polity_dist
0,1990,ARG,AGO,0,0,7977.915823,32.580860,141352.3800,4338.51000,AR,...,0,24.747800,1,NaN,Argentina,Angola,ARG-AGO,7.0,-7.0,14.0
1,1990,AUS,AGO,0,0,12954.905818,17.065100,319264.7000,18708.63300,AU,...,1,1.296240,1,NaN,Australia,Angola,AUS-AGO,10.0,-7.0,17.0
2,1990,AUT,AGO,0,0,6432.058253,7.710882,164984.0300,21396.26000,AT,...,1,2.999282,1,NaN,Austria,Angola,AUT-AGO,10.0,-7.0,17.0
3,1990,BDI,AGO,0,0,1905.419703,5.670250,1132.1012,199.65631,BI,...,0,0.000000,1,NaN,Burundi,Angola,BDI-AGO,-6.0,-7.0,1.0
4,1990,BEL,AGO,0,0,6817.662686,9.967400,202690.9700,20335.39000,BE,...,1,76.786900,1,NaN,Belgium,Angola,BEL-AGO,10.0,-7.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334437,2002,TWN,LUX,0,0,9609.759792,22.400000,294803.0000,13160.84900,TW,...,0,637.017600,0,NaN,Taiwan,Luxembourg,TWN-LUX,9.0,10.0,1.0
334438,2003,TWN,LUX,0,0,9609.759792,22.490000,299785.0000,13329.70200,TW,...,0,775.699000,0,NaN,Taiwan,Luxembourg,TWN-LUX,9.0,10.0,1.0
334439,2004,TWN,LUX,0,0,9609.759792,22.580000,322179.0000,14268.33500,TW,...,0,485.395600,0,NaN,Taiwan,Luxembourg,TWN-LUX,10.0,10.0,0.0
334440,2005,TWN,LUX,0,0,9609.759792,22.650000,345862.0000,15269.84600,TW,...,0,248.613600,0,NaN,Taiwan,Luxembourg,TWN-LUX,10.0,10.0,0.0


In [3]:
data.columns

Index(['year', 'iso_o', 'iso_d', 'contig', 'comlang_off', 'distw', 'pop_o',
       'gdp_o', 'gdpcap_o', 'iso2_o', 'pop_d', 'gdp_d', 'gdpcap_d', 'iso2_d',
       'heg_d', 'conflict', 'indepdate', 'heg_o', 'col_to', 'col_fr',
       'col_hist', 'col_cur', 'sever', 'sib_conflict', 'gatt_o', 'gatt_d',
       'rta', 'comleg', 'comcur', 'acp_to_eu', 'gsp', 'eu_to_acp', 'gsp_rec',
       'flow', 'validmirror', 'family', 'country_o', 'country_d', 'tradelink',
       'polity_o', 'polity_d', 'polity_dist'],
      dtype='object')

In [4]:
# Drop NaN in polity_dist column
data = data.dropna(subset=['polity_dist'])
data.shape

(325396, 42)

In [5]:
data.loc[:,'log_gdp_o'] = np.log(data.gdp_o)
data.loc[:,'log_gdp_d'] = np.log(data.gdp_d)
data.loc[:,'log_flow'] = np.log(data.flow)
data.loc[:,'log_flow'] = data.loc[:,'log_flow'].apply(lambda x: np.log(1+x))
data.loc[:,'log_distw'] = data.loc[:,'distw'].apply(lambda x: np.log(1+x))
data.loc[:,'log_polity_dist'] = data.loc[:,'polity_dist'].apply(lambda x: np.log(int(1+x)))
data

/home/topher-lo/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/topher-lo/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/topher-lo/.local/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,country_d,tradelink,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist
0,1990,ARG,AGO,0,0,7977.915823,32.580860,141352.3800,4338.51000,AR,...,Angola,ARG-AGO,7.0,-7.0,14.0,11.859011,9.236027,1.437163,8.984558,2.708050
1,1990,AUS,AGO,0,0,12954.905818,17.065100,319264.7000,18708.63300,AU,...,Angola,AUS-AGO,10.0,-7.0,17.0,12.673776,9.236027,0.230689,9.469307,2.890372
2,1990,AUT,AGO,0,0,6432.058253,7.710882,164984.0300,21396.26000,AT,...,Angola,AUT-AGO,10.0,-7.0,17.0,12.013604,9.236027,0.741162,8.769205,2.890372
3,1990,BDI,AGO,0,0,1905.419703,5.670250,1132.1012,199.65631,BI,...,Angola,BDI-AGO,-6.0,-7.0,1.0,7.031831,9.236027,NaN,7.552982,0.693147
4,1990,BEL,AGO,0,0,6817.662686,9.967400,202690.9700,20335.39000,BE,...,Angola,BEL-AGO,10.0,-7.0,17.0,12.219438,9.236027,1.675419,8.827419,2.890372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334437,2002,TWN,LUX,0,0,9609.759792,22.400000,294803.0000,13160.84900,TW,...,Luxembourg,TWN-LUX,9.0,10.0,1.0,12.594063,10.026265,2.009126,9.170639,0.693147
334438,2003,TWN,LUX,0,0,9609.759792,22.490000,299785.0000,13329.70200,TW,...,Luxembourg,TWN-LUX,9.0,10.0,1.0,12.610821,10.274654,2.035198,9.170639,0.693147
334439,2004,TWN,LUX,0,0,9609.759792,22.580000,322179.0000,14268.33500,TW,...,Luxembourg,TWN-LUX,10.0,10.0,0.0,12.682863,10.422069,1.971991,9.170639,0.000000
334440,2005,TWN,LUX,0,0,9609.759792,22.650000,345862.0000,15269.84600,TW,...,Luxembourg,TWN-LUX,10.0,10.0,0.0,12.753795,10.504210,1.874245,9.170639,0.000000


In [6]:
# Set entity-time multi index

data = data.set_index(['tradelink', 'year'])

In [7]:
# Drop rows with NaN in ['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_flow']

data = data.dropna(subset=['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_flow'])

In [8]:
# Remaining countries after dropping NaN
print('Number of countries remaining', len(data['country_o'].unique()))
data['country_o'].unique()

Number of countries remaining 156


array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada',
       'Switzerland', 'China', 'Germany', 'Denmark', 'Spain', 'Finland',
       'France', 'United Kingdom', 'Greece', 'Hungary', 'Indonesia',
       'India', 'Ireland', 'Italy', 'Japan', 'Kenya', 'Korea South',
       'Mexico', 'Malaysia', 'Netherlands', 'Norway', 'Philippines',
       'Poland', 'Portugal', 'Sweden', 'Thailand', 'Tunisia', 'Turkey',
       'Taiwan', 'Uruguay', 'United States', 'Zambia', 'Zimbabwe',
       'Bangladesh', 'Bahrain', 'Chile', 'Cyprus', 'Egypt', 'Iran',
       'Iraq', 'Jordan', 'Libya', 'Sri Lanka', 'Morocco', 'Mozambique',
       'New Zealand', 'Pakistan', 'Romania', 'Saudi Arabia', 'Singapore',
       'Somalia', 'Tanzania', 'Yemen', 'Oman', 'Qatar', 'Syria',
       'Bulgaria', 'Cameroon', 'Colombia', 'Costa Rica', 'Cuba',
       'Algeria', 'Guatemala', 'Guyana', 'Israel', 'Jamaica', 'Mauritius',
       'Malawi', 'Peru', 'Papua New Guinea', 'El Salvador', 'Venezuela',
       'Vietna

In [9]:
# Remaining countries number of tradelinks after dropping NaN
data['country_o'].value_counts

<bound method IndexOpsMixin.value_counts of tradelink  year
ARG-AGO    1990    Argentina
AUS-AGO    1990    Australia
AUT-AGO    1990      Austria
BEL-AGO    1990      Belgium
BRA-AGO    1990       Brazil
                     ...    
TWN-LUX    2001       Taiwan
           2002       Taiwan
           2003       Taiwan
           2004       Taiwan
           2005       Taiwan
Name: country_o, Length: 171920, dtype: object>

## 2. Results
---

In [10]:
model_basic = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,log_flow,R-squared:,0.3985
Estimator:,PanelOLS,R-squared (Between):,-0.9731
No. Observations:,171920,R-squared (Within):,0.0511
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.2312
Time:,00:54:12,Log-likelihood,-1.916e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,3.797e+04
Entities:,21360,P-value,0.0000
Avg Obs:,8.0487,Distribution:,"F(3,171900)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,6117.9


In [11]:
model_modified = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3987
Estimator:,PanelOLS,R-squared (Between):,-0.9137
No. Observations:,171920,R-squared (Within):,0.0510
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.1903
Time:,00:54:13,Log-likelihood,-1.916e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.85e+04
Entities:,21360,P-value,0.0000
Avg Obs:,8.0487,Distribution:,"F(4,171899)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,4547.6


In [12]:
model_modified_control = PanelOLS(data.log_flow, data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist','col_hist']], time_effects=True)
result_modified_control = model_modified_control.fit(cov_type='clustered', cluster_entity=True)
result_modified_control.summary

Dep. Variable:,log_flow,R-squared:,0.4013
Estimator:,PanelOLS,R-squared (Between):,-0.9070
No. Observations:,171920,R-squared (Within):,0.0511
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.1864
Time:,00:54:14,Log-likelihood,-1.913e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.304e+04
Entities:,21360,P-value,0.0000
Avg Obs:,8.0487,Distribution:,"F(5,171898)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,3621.7


In [13]:
post_89_data = data.query("year >= 1989")

In [14]:
model_basic = PanelOLS(post_89_data.log_flow, post_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,log_flow,R-squared:,0.3985
Estimator:,PanelOLS,R-squared (Between):,-0.9731
No. Observations:,171920,R-squared (Within):,0.0511
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.2312
Time:,00:54:15,Log-likelihood,-1.916e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,3.797e+04
Entities:,21360,P-value,0.0000
Avg Obs:,8.0487,Distribution:,"F(3,171900)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,6117.9


In [15]:
model_modified = PanelOLS(post_89_data.log_flow, post_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3987
Estimator:,PanelOLS,R-squared (Between):,-0.9137
No. Observations:,171920,R-squared (Within):,0.0510
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.1903
Time:,00:54:16,Log-likelihood,-1.916e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.85e+04
Entities:,21360,P-value,0.0000
Avg Obs:,8.0487,Distribution:,"F(4,171899)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,4547.6


In [16]:
# World post-1989 exclude China
post_89_noChina_data = data.query("year >= 1989")
post_89_noChina_data = post_89_noChina_data[post_89_noChina_data["iso_o"] != ("CHN" or "USA")]
post_89_noChina_data = post_89_noChina_data[post_89_noChina_data["iso_d"] != ("CHN" or "USA")]

In [17]:
model_basic = PanelOLS(post_89_noChina_data.log_flow, post_89_noChina_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,log_flow,R-squared:,0.3945
Estimator:,PanelOLS,R-squared (Between):,-0.9508
No. Observations:,167434,R-squared (Within):,0.0479
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.2102
Time:,00:54:18,Log-likelihood,-1.877e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,3.637e+04
Entities:,21360,P-value,0.0000
Avg Obs:,7.8387,Distribution:,"F(3,167414)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,5908.5


In [18]:
model_modified = PanelOLS(post_89_noChina_data.log_flow, post_89_noChina_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3948
Estimator:,PanelOLS,R-squared (Between):,-0.8839
No. Observations:,167434,R-squared (Within):,0.0478
Date:,"Thu, Nov 26 2020",R-squared (Overall):,-0.1643
Time:,00:54:19,Log-likelihood,-1.877e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.73e+04
Entities:,21360,P-value,0.0000
Avg Obs:,7.8387,Distribution:,"F(4,167413)"
Min Obs:,0.0000,,
Max Obs:,18.000,F-statistic (robust):,4392.0


In [19]:
# Dataset of former Soviet countries as the origin from 1960-2006
FORMER_SOVIET_COUNTRIES = {
    "Estonia": "EST",
    "Latvia": "LVA",
    "Lithuania": "LTU",
    "Kazakhstan": "KAZ",
    "Kyrgyzstan": "KGZ",
    "Tajikistan": "TJK",
    "Turkmenistan": "TKM",
    "Uzbekistan": "UZB",
    "Belarus": "BLR",
    "Moldova": "MDA",
    "Ukraine": "UKR",
    "Russia": "RUS",
    "Armenia": "ARM",
    "Azerbaijan": "AZE",
    "Georgia": "GEO"
}
post_soviet_data = data
post_soviet_data = post_soviet_data[post_soviet_data['iso_o'].isin(list(FORMER_SOVIET_COUNTRIES.values()))]
post_soviet_data.head(10)

,,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,pop_d,...,country_o,country_d,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist
tradelink,year,,,,,,,,,,,,,,,,,,,,,
RUS-AGO,1992,RUS,AGO,0,0,8103.861455,148.689000,460205.4400,3095.08740,RU,11.203767,...,Russia,Angola,5.0,0.0,5.0,13.039428,8.662054,1.470073,9.000219,1.791759
RUS-ARE,1992,RUS,ARE,0,0,3721.569046,148.689000,460205.4400,3095.08740,RU,1.983000,...,Russia,UAE,5.0,-8.0,13.0,13.039428,10.474826,1.761459,8.222169,2.639057
AZE-AUS,1992,AZE,AUS,0,0,12862.659951,7.382000,4991.3500,676.15150,AZ,17.495000,...,Azerbaijan,Australia,1.0,10.0,9.0,8.515462,12.681400,0.784793,9.462162,2.302585
RUS-AUS,1992,RUS,AUS,0,0,13424.896701,148.689000,460205.4400,3095.08740,RU,17.495000,...,Russia,Australia,5.0,10.0,5.0,13.039428,12.681400,1.151259,9.504941,1.791759
BLR-AUS,1992,BLR,AUS,0,0,14713.924563,10.216000,17022.1800,1666.22750,BY,17.495000,...,Belarus,Australia,6.0,10.0,4.0,9.742272,12.681400,-0.131073,9.596618,1.609438
AZE-AUT,1992,AZE,AUT,0,0,2790.650554,7.382000,4991.3500,676.15150,AZ,7.782519,...,Azerbaijan,Austria,1.0,10.0,9.0,8.515462,12.179133,1.528359,7.934388,2.302585
EST-AUT,1992,EST,AUT,0,0,1421.983529,1.533000,3986.6152,2600.53170,EE,7.782519,...,Estonia,Austria,6.0,10.0,4.0,8.290698,12.179133,0.646680,7.260511,1.609438
MDA-AUT,1992,MDA,AUT,0,0,1026.710341,4.410836,2319.2434,525.80585,MD,7.782519,...,Moldova,Austria,5.0,10.0,5.0,7.748996,12.179133,0.095260,6.935089,1.791759
RUS-AUT,1992,RUS,AUT,0,0,2527.122805,148.689000,460205.4400,3095.08740,RU,7.782519,...,Russia,Austria,5.0,10.0,5.0,13.039428,12.179133,2.025908,7.835232,1.791759


In [20]:
model_basic = PanelOLS(post_soviet_data.log_flow, post_soviet_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,log_flow,R-squared:,0.3284
Estimator:,PanelOLS,R-squared (Between):,0.5470
No. Observations:,13576,R-squared (Within):,0.0548
Date:,"Thu, Nov 26 2020",R-squared (Overall):,0.6259
Time:,00:54:19,Log-likelihood,-1.581e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,2209.5
Entities:,20464,P-value,0.0000
Avg Obs:,0.6634,Distribution:,"F(3,13558)"
Min Obs:,0.0000,,
Max Obs:,16.000,F-statistic (robust):,675.00


In [21]:
model_modified = PanelOLS(post_soviet_data.log_flow, post_soviet_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3287
Estimator:,PanelOLS,R-squared (Between):,0.5508
No. Observations:,13576,R-squared (Within):,0.0542
Date:,"Thu, Nov 26 2020",R-squared (Overall):,0.6326
Time:,00:54:20,Log-likelihood,-1.58e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1659.2
Entities:,20464,P-value,0.0000
Avg Obs:,0.6634,Distribution:,"F(4,13557)"
Min Obs:,0.0000,,
Max Obs:,16.000,F-statistic (robust):,507.53


In [22]:
post_soviet_89_data = post_soviet_data.query("year >= 1989")
post_soviet_89_data.head()

,,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,pop_d,...,country_o,country_d,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist
tradelink,year,,,,,,,,,,,,,,,,,,,,,
RUS-AGO,1992,RUS,AGO,0,0,8103.861455,148.689,460205.44,3095.0874,RU,11.203767,...,Russia,Angola,5.0,0.0,5.0,13.039428,8.662054,1.470073,9.000219,1.791759
RUS-ARE,1992,RUS,ARE,0,0,3721.569046,148.689,460205.44,3095.0874,RU,1.983000,...,Russia,UAE,5.0,-8.0,13.0,13.039428,10.474826,1.761459,8.222169,2.639057
AZE-AUS,1992,AZE,AUS,0,0,12862.659951,7.382,4991.35,676.1515,AZ,17.495000,...,Azerbaijan,Australia,1.0,10.0,9.0,8.515462,12.681400,0.784793,9.462162,2.302585
RUS-AUS,1992,RUS,AUS,0,0,13424.896701,148.689,460205.44,3095.0874,RU,17.495000,...,Russia,Australia,5.0,10.0,5.0,13.039428,12.681400,1.151259,9.504941,1.791759
BLR-AUS,1992,BLR,AUS,0,0,14713.924563,10.216,17022.18,1666.2275,BY,17.495000,...,Belarus,Australia,6.0,10.0,4.0,9.742272,12.681400,-0.131073,9.596618,1.609438


In [23]:
model_basic = PanelOLS(post_soviet_89_data.log_flow, post_soviet_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw']], time_effects=True)
result_basic = model_basic.fit(cov_type='clustered', cluster_entity=True)
result_basic.summary

/home/topher-lo/anaconda3/envs/trade-polity/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,log_flow,R-squared:,0.3284
Estimator:,PanelOLS,R-squared (Between):,0.5470
No. Observations:,13576,R-squared (Within):,0.0548
Date:,"Thu, Nov 26 2020",R-squared (Overall):,0.6259
Time:,00:54:20,Log-likelihood,-1.581e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,2209.5
Entities:,20464,P-value,0.0000
Avg Obs:,0.6634,Distribution:,"F(3,13558)"
Min Obs:,0.0000,,
Max Obs:,16.000,F-statistic (robust):,675.00


In [24]:
model_modified = PanelOLS(post_soviet_89_data.log_flow, post_soviet_89_data[['log_gdp_o', 'log_gdp_d', 'log_distw', 'log_polity_dist']], time_effects=True)
result_modified = model_modified.fit(cov_type='clustered', cluster_entity=True)
result_modified.summary

Dep. Variable:,log_flow,R-squared:,0.3287
Estimator:,PanelOLS,R-squared (Between):,0.5508
No. Observations:,13576,R-squared (Within):,0.0542
Date:,"Thu, Nov 26 2020",R-squared (Overall):,0.6326
Time:,00:54:20,Log-likelihood,-1.58e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1659.2
Entities:,20464,P-value,0.0000
Avg Obs:,0.6634,Distribution:,"F(4,13557)"
Min Obs:,0.0000,,
Max Obs:,16.000,F-statistic (robust):,507.53
